In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode,split
spark = SparkSession.builder.appName('TwitterStream').getOrCreate()
spark

In [7]:
lines = spark.readStream.format("kafka").option("host","kafka-broker:9092").option("subscribe",'test').load()
lines.selectExpr("CAST(value AS STRING)")
words = lines.select(explode(split(lines.value, ' ')).alias('word'))
hashtags = words.filter(words['word'].startswith('#'))

AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of "Structured Streaming + Kafka Integration Guide".;

In [5]:
res = hashtags.groupBy('word').count()
res = res.orderBy(res['count'].desc())

In [6]:
query = res.writeStream.outputMode('complete').format('console').start()
#query.awaitTermination()